In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

data_parsing = pd.read_excel('data/Data_TSUM.xlsx', sheet_name=0)
data_company = pd.read_excel('data/Data_TSUM.xlsx', sheet_name=1)
display('Parsing',data_parsing.head(),'Company',
data_company.head())

'Parsing'

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa1111_11,black,167
1,Valentino,Shoes,aaaaa1111_12,black,188
2,Valentino,Shoes,aaaaa1111_13,black,184
3,Valentino,Shoes,aaaaa1111_14,bla//ck,196
4,Valentino,Shoes,aaaaa1111_15,bla\\ck,250


'Company'

,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,153


In [3]:
symbols = ['_', '-', '~', '\\', '/', '//', '/\\']
columns_parsing_to_clean = ['producer_id', 'producer_color']

def str_replace_list(string):
    result = string
    for sym in symbols:
        result = result.replace(sym, '')
    return result

for column in columns_parsing_to_clean:
    data_parsing[column] = data_parsing[column].apply(str_replace_list)

In [4]:
display('Parsing',data_parsing.head(),'Company',
data_company.head())

'Parsing'

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa111111,black,167
1,Valentino,Shoes,aaaaa111112,black,188
2,Valentino,Shoes,aaaaa111113,black,184
3,Valentino,Shoes,aaaaa111114,black,196
4,Valentino,Shoes,aaaaa111115,black,250


'Company'

,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,153


In [5]:
data_concat = pd.concat([data_parsing, data_company[['item_id', 'color_id', 'current price']]], axis=1)
data_concat.head()

,brand,Category,producer_id,producer_color,price,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153


In [6]:
data_concat['difference'] = data_concat['price'] / data_concat['current price'] * 100
data_concat.head()

,brand,Category,producer_id,producer_color,price,item_id,color_id,current price,difference
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,67.611336
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,116.770186
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,78.632479
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,117.365269
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,163.398693


In [7]:
data_concat[data_concat['brand'] == 'Valentino']['difference'].describe()

count     25.000000
mean     101.358873
std       32.820943
min       58.020478
25%       72.075472
50%       94.800000
75%      117.365269
max      168.421053
Name: difference, dtype: float64

In [8]:
def outliers_iqr(data:pd.DataFrame, feature, left=1.5, right=1.5, log_scale=False, log_adder=0):
    if log_scale:
        x = np.log(data[feature] + log_adder)
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
    

In [12]:
def outliers_z_score(data:pd.DataFrame, feature,
                    log_scale=False, left=3, right=3, log_adder=0):
    if log_scale:
        x = np.log(data[feature] + log_adder)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left*sigma
    upper_bound = mu + right*sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned 

In [11]:
outliers, cleaned = outliers_iqr(data_concat, 'difference')
outliers.shape

(0, 9)

In [13]:
outliers, cleaned = outliers_z_score(data_concat, 'difference')
outliers.shape

(0, 9)

In [22]:
data_concat['brand'].value_counts()

Valentino       25
Brioni          25
Stone Island    25
Name: brand, dtype: int64

In [23]:
data_plot_valentino = data_concat[data_concat['brand'] == 'Valentino'].copy()
#data_plot = data_plot['difference'].apply(lambda x: )
plot_valentino = px.histogram(
    data_plot_valentino,
    x='difference'
)
plot_valentino.show()
data_plot_brioni = data_concat[data_concat['brand'] == 'Brioni'].copy()
plot_brioni = px.histogram(
    data_plot_brioni,
    x='difference'
)
plot_brioni.show()
data_plot_stone = data_concat[data_concat['brand'] == 'Stone Island'].copy()
plot_stone = px.histogram(
    data_plot_stone,
    x='difference'
)
plot_stone.show()


In [24]:
data_valentino = data_concat[data_concat['brand'] == 'Valentino']
data_brioni = data_concat[data_concat['brand'] == 'Brioni']
data_stone = data_concat[data_concat['brand'] == 'Stone Island']

In [26]:
outliers_val_iqr, cleaned_val_iqr = outliers_iqr(data_valentino, 'difference')
outliers_val_z, cleaned_val_z = outliers_z_score(data_valentino, 'difference')
display(
    'valentino iqr outliers',
    outliers_val_iqr.shape[0],
    'valentino z outliers',
    outliers_val_z.shape[0]
)

'valentino iqr outliers'

0

'valentino z outliers'

0

In [27]:
outliers_brio_iqr, cleaned_brio_iqr = outliers_iqr(data_brioni, 'difference')
outliers_brio_z, cleaned_brio_z = outliers_z_score(data_brioni, 'difference')
display(
    'brioni iqr outliers',
    outliers_brio_iqr.shape[0],
    'brioni z outliers',
    outliers_brio_z.shape[0]
)

'brioni iqr outliers'

0

'brioni z outliers'

0

In [28]:
outliers_stone_iqr, cleaned_stone_iqr = outliers_iqr(data_stone, 'difference')
outliers_stone_z, cleaned_stone_z = outliers_z_score(data_stone, 'difference')
display(
    'stone iqr outliers',
    outliers_stone_iqr.shape[0],
    'stone z outliers',
    outliers_stone_z.shape[0]
)

'stone iqr outliers'

0

'stone z outliers'

0

In [29]:
outliers_val_iqr, cleaned_val_iqr = outliers_iqr(data_valentino, 'difference', log_scale=True)
outliers_val_z, cleaned_val_z = outliers_z_score(data_valentino, 'difference', log_scale=True)
display(
    'valentino iqr outliers',
    outliers_val_iqr.shape[0],
    'valentino z outliers',
    outliers_val_z.shape[0]
)

'valentino iqr outliers'

0

'valentino z outliers'

0

In [30]:
outliers_brio_iqr, cleaned_brio_iqr = outliers_iqr(data_brioni, 'difference', log_scale=True)
outliers_brio_z, cleaned_brio_z = outliers_z_score(data_brioni, 'difference', log_scale=True)
display(
    'brioni iqr outliers',
    outliers_brio_iqr.shape[0],
    'brioni z outliers',
    outliers_brio_z.shape[0]
)

'brioni iqr outliers'

0

'brioni z outliers'

0

In [31]:
outliers_stone_iqr, cleaned_stone_iqr = outliers_iqr(data_stone, 'difference', log_scale=True)
outliers_stone_z, cleaned_stone_z = outliers_z_score(data_stone, 'difference', log_scale=True)
display(
    'stone iqr outliers',
    outliers_stone_iqr.shape[0],
    'stone z outliers',
    outliers_stone_z.shape[0]
)

'stone iqr outliers'

0

'stone z outliers'

0